# Observations

- period and calculation_type features all the same value, can drop
- split time series into batches of shorter sequences
- how to impute intermediate missing values? -> take last? interpolate?
- add year, month, absolute int as features for date

- one hot encoding of categorical features: too many, not important enough... drop? keep as single numerical feature?

- duplicate datapoints, same brand, same date, same values - two different compsets...

# TODO:
- align the data by brand
- are there weeks missing in between?
- normalize values


# Discussion
- should we predict by brand or by legal entity?


- standardization in the end after adding additional metrics

In [1]:
import pandas as pd 

In [51]:
DATA_PATH = "/home/david/code/datathon24-personal/data/skylab_instagram_datathon_dataset.csv"

df = pd.read_csv(DATA_PATH, sep=";")


def video_to_picture_ratio(df, videos_col, pictures_col):
    df['video_picture_ratio'] = df[videos_col] / df[pictures_col]
    return df


def calculate_rolling_average_per_brand(data, brand_column, window_size=7):
    """
    Calculate the rolling average of 'engagement' for each brand in a DataFrame.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        brand_column (str): The column name which identifies the brand.
        window_size (int): The number of observations used for calculating the rolling average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the rolling average of 'engagement' calculated per brand.
    """
    # Group by brand and calculate rolling average within each group
    data['rolling_avg_engagement'] = data.groupby(brand_column)['engagement'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    return data


def calculate_exponential_moving_average_per_brand(data, brand_column, span=7):
    """
    Calculate the exponential moving average of 'engagement' for each brand in a DataFrame.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        brand_column (str): The column name which identifies the brand.
        span (int): The decay in terms of the span of the exponential window.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the exponential moving average of 'engagement' calculated per brand.
    """
    # Group by brand and calculate EMA within each group
    data['ewma_engagement'] = data.groupby(brand_column)['engagement'].transform(lambda x: x.ewm(span=span, adjust=False).mean())
    return data


def calculate_brand_wise_growth_rates(data, column_names, brand_column):
    """
    Calculate the growth rate for specified columns in a DataFrame, grouped by brand.
    Replace NaN values with 0 where growth rate cannot be calculated.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        column_names (list of str): A list of column names to calculate the growth rate.
        brand_column (str): The column name which identifies the brand.

    Returns:
        pd.DataFrame: The DataFrame with new columns for each specified column's growth rate, calculated for each brand and NaN replaced by 0.
    """
    for column in column_names:
        # Calculate growth rate within each brand group
        data[f'growth_rate_{column}'] = data.groupby(brand_column)[column].pct_change() * 100
        # Replace NaN values with 0
        data[f'growth_rate_{column}'].fillna(0, inplace=True)
    return data

def calculate_brand_rolling_statistics(data, column_name, brand_column, window_size=7):
    """
    Calculate rolling statistics for a specified column in a DataFrame, grouped by brand,
    and replace NaN values with 0.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The column name to calculate rolling statistics.
        brand_column (str): The column name which identifies the brand.
        window_size (int): The number of observations used for calculating the rolling statistic.

    Returns:
        pd.DataFrame: The DataFrame with new columns for each rolling statistic of the specified column, calculated for each brand.
    """
    grouped = data.groupby(brand_column)[column_name]
    data[f'{column_name}_rolling_min'] = grouped.transform(lambda x: x.rolling(window=window_size, min_periods=1).min()).fillna(0)
    data[f'{column_name}_rolling_max'] = grouped.transform(lambda x: x.rolling(window=window_size, min_periods=1).max()).fillna(0)
    data[f'{column_name}_rolling_std'] = grouped.transform(lambda x: x.rolling(window=window_size, min_periods=1).std()).fillna(0)

    return data

def create_brand_lag_features(data, column_name, brand_column, lag_periods):
    """
    Create lag features for a specified column in a DataFrame, grouped by brand,
    and replace NaN values with 0.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The column name to create lag features for.
        brand_column (str): The column name which identifies the brand.
        lag_periods (int): The number of lag periods.

    Returns:
        pd.DataFrame: The DataFrame with new columns for each lag feature of the specified column, calculated for each brand.
    """
    for i in range(1, lag_periods + 1):
        data[f'{column_name}_lag_{i}'] = data.groupby(brand_column)[column_name].shift(i).fillna(0)

    return data


def preprocess_data(df, missing_values_cutoff=0.7, test_fraction=0.2):

    df = df.drop(columns=["period", "calculation_type", "compset", "compset_group", "legal_entity_name", "ultimate_parent_legal_entity_name", "primary_exchange_name"])
    df["period_end_date"] = pd.to_datetime(df["period_end_date"])

    df = df.rename(columns={'business_entity_doing_business_as_name': 'brand', 'period_end_date': 'date'})

    df = df[df['brand'] != "All Brands"]
    df = df[df['brand'] != "Boca"]

    df = df.groupby(['brand', 'date']).first().reset_index()

    def correct_country_name(name):
        country_map = {
            "Hong Kong": "China",
            "China;Hong Kong": "China",
            ";France": "France",
            ";": None,
            "Belgium;": "Belgium"
        }
        if name in country_map.keys():
            return country_map[name]
        return name

    df['domicile_country_name'] = df['domicile_country_name'].apply(correct_country_name)


    categorical_features = ["domicile_country_name"]
    for feature in categorical_features:
        df = pd.get_dummies(df, columns=[feature], prefix=feature, dummy_na=True, dtype=int)

    # TODO: remove bad data
    na_frac = df[['brand', 'followers', 'pictures',
        'videos', 'comments', 'likes']].groupby('brand').apply(lambda x: x.iloc[:,1:].isna().sum()/len(x))

    bad_brands = list(na_frac[na_frac.max(axis=1) > missing_values_cutoff].index)

    df = df[~df['brand'].isin(bad_brands)]


    # TODO: add additional features
    df['engagement'] = df['comments'] + df['likes']
    df['engagement_rate'] = df['engagement']/df['followers']
    df['engagement_rate_per_post'] =  df['engagement_rate']/(df['videos'] + df['pictures'])


    # df = video_to_picture_ratio(df, "videos", "pictures")
    df = calculate_rolling_average_per_brand(df, "brand", window_size=4)
    df = calculate_exponential_moving_average_per_brand(df, "brand", span=4)
    # df = calculate_brand_wise_growth_rates(df, ["comments", "likes", "followers", "pictures", "videos"], "brand")
    for c in ["comments", "likes", "followers", "pictures", "videos"]:
        df = calculate_brand_rolling_statistics(df, c, "brand", window_size=4)
        df = create_brand_lag_features(df, c, "brand", 4)

    
    # Step 1: Sort the dataframe by time
    df_sorted = df.sort_values(by='date')

    # Step 2: Group the dataframe by 'Brand'
    grouped = df_sorted.groupby('brand')

    # Step 3: Define an empty dataframe for train and test sets
    train_df = pd.DataFrame(columns=df.columns)  # Columns same as original dataframe
    test_df = pd.DataFrame(columns=df.columns)   # Columns same as original dataframe

    # Step 4: Iterate over each group and split into train and test sets
    for _, group in grouped:
        n_rows = len(group)
        n_test = int(test_fraction * n_rows)  # 20% of rows for test set

        # Add last 20% of rows to test set
        test_df = pd.concat([test_df, group.iloc[-n_test:]])

        # Add remaining rows to train set
        train_df = pd.concat([train_df, group.iloc[:-n_test]])

    # Step 5: Reset index for both train and test dataframes
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)


    # TODO: normalize values

    normalize_cols = [c for c in train_df.columns if c not in ['brand', 'date'] and not c.startswith("domicile_country_name")]
    for col in normalize_cols:
        m = train_df[col].mean()
        s = train_df[col].std()
        train_df[col] = (train_df[col] - m)/s
        test_df[col] = (test_df[col] - m)/s


    # TODO: impute missing values
  
    # TODO: impute missing values
    train_df = train_df.groupby('brand').apply(lambda group: group.fillna(method='ffill'))
    train_df = train_df.reset_index(drop=True)

    train_df = train_df.groupby('brand').apply(lambda group: group.fillna(method='bfill'))
    train_df = train_df.reset_index(drop=True)


    test_df = test_df.groupby('brand').apply(lambda group: group.fillna(method='ffill'))
    test_df = test_df.reset_index(drop=True)

    test_df = test_df.groupby('brand').apply(lambda group: group.fillna(method='bfill'))
    test_df = test_df.reset_index(drop=True)



    return train_df, test_df


def prepare_data_lstm(df, sequence_length=10, prediction_dist=4, missing_values_cutoff=0.7, test_fraction=0.2):

    df = df.drop(columns=["period", "calculation_type", "compset", "compset_group", "legal_entity_name", "ultimate_parent_legal_entity_name", "primary_exchange_name"])
    df["period_end_date"] = pd.to_datetime(df["period_end_date"])

    df = df.rename(columns={'business_entity_doing_business_as_name': 'brand', 'period_end_date': 'date'})

    df = df[df['brand'] != "All Brands"]
    df = df[df['brand'] != "Boca"]

    df = df.groupby(['brand', 'date']).first().reset_index()

    def correct_country_name(name):
        country_map = {
            "Hong Kong": "China",
            "China;Hong Kong": "China",
            ";France": "France",
            ";": None,
            "Belgium;": "Belgium"
        }
        if name in country_map.keys():
            return country_map[name]
        return name

    df['domicile_country_name'] = df['domicile_country_name'].apply(correct_country_name)


    categorical_features = ["domicile_country_name"]
    for feature in categorical_features:
        df = pd.get_dummies(df, columns=[feature], prefix=feature, dummy_na=True, dtype=int)

    # TODO: remove bad data
    na_frac = df[['brand', 'followers', 'pictures',
        'videos', 'comments', 'likes']].groupby('brand').apply(lambda x: x.iloc[:,1:].isna().sum()/len(x))

    bad_brands = list(na_frac[na_frac.max(axis=1) > missing_values_cutoff].index)

    df = df[~df['brand'].isin(bad_brands)]


    # TODO: add additional features
    df['engagement'] = df['comments'] + df['likes']
    df['engagement_rate'] = df['engagement']/df['followers']
    df['engagement_rate_per_post'] =  df['engagement_rate']/(df['videos'] + df['pictures'])


    # df = video_to_picture_ratio(df, "videos", "pictures")
    df = calculate_rolling_average_per_brand(df, "brand", window_size=4)
    df = calculate_exponential_moving_average_per_brand(df, "brand", span=4)
    # df = calculate_brand_wise_growth_rates(df, ["comments", "likes", "followers", "pictures", "videos"], "brand")
    for c in ["comments", "likes", "followers", "pictures", "videos"]:
        df = calculate_brand_rolling_statistics(df, c, "brand", window_size=4)
        df = create_brand_lag_features(df, c, "brand", 4)



    # Step 1: Sort the dataframe by time
    df_sorted = df.sort_values(by='date')

    # Step 2: Group the dataframe by 'Brand'
    grouped = df_sorted.groupby('brand')

    # Step 3: Define an empty dataframe for train and test sets
    train_df = pd.DataFrame(columns=df.columns)  # Columns same as original dataframe
    test_df = pd.DataFrame(columns=df.columns)   # Columns same as original dataframe

    # Step 4: Iterate over each group and split into train and test sets
    for _, group in grouped:
        n_rows = len(group)
        n_test = int(test_fraction * n_rows)  # 20% of rows for test set

        # Add last 20% of rows to test set
        test_df = pd.concat([test_df, group.iloc[-n_test:]])

        # Add remaining rows to train set
        train_df = pd.concat([train_df, group.iloc[:-n_test]])

    # Step 5: Reset index for both train and test dataframes
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)


    # TODO: normalize values

    normalize_cols = [c for c in train_df.columns if c not in ['brand', 'date'] and not c.startswith("domicile_country_name")]
    for col in normalize_cols:
        m = train_df[col].mean()
        s = train_df[col].std()
        train_df[col] = (train_df[col] - m)/s
        test_df[col] = (test_df[col] - m)/s


    # TODO: impute missing values
  
    # TODO: impute missing values
    train_df = train_df.groupby('brand').apply(lambda group: group.fillna(method='ffill'))
    train_df = train_df.reset_index(drop=True)

    train_df = train_df.groupby('brand').apply(lambda group: group.fillna(method='bfill'))
    train_df = train_df.reset_index(drop=True)


    test_df = test_df.groupby('brand').apply(lambda group: group.fillna(method='ffill'))
    test_df = test_df.reset_index(drop=True)

    test_df = test_df.groupby('brand').apply(lambda group: group.fillna(method='bfill'))
    test_df = test_df.reset_index(drop=True)


    # TODO: create sequences

    # TODO: split df into brands
    # sequence per brand
    # label

    label_col = "engagement_rate_per_post"

    train_sequences = []
    train_labels = []
    test_sequences = []
    test_labels = []


    for bi, brand in enumerate(df['brand'].unique()):

        brand_train_df = train_df[train_df['brand'] == brand]

        cols = [c for c in train_df.columns if (c != "brand" and c != "date")]
        brand_train_df = brand_train_df[cols]

        for i in range(len(brand_train_df) - (sequence_length + prediction_dist)):
            sequence = brand_train_df.iloc[i:i + sequence_length].values
            train_sequences.append(sequence)

            sequence_labels = brand_train_df.iloc[i+sequence_length+prediction_dist][label_col]
            train_labels.append(sequence_labels)
        
        brand_test_df = test_df[test_df['brand'] == brand]

        brand_test_df = brand_test_df[cols]


        for i in range(len(brand_test_df) - (sequence_length + prediction_dist)):
            sequence = brand_test_df.iloc[i:i + sequence_length].values
            test_sequences.append(sequence)

            sequence_labels = brand_test_df.iloc[i+sequence_length+prediction_dist][label_col]
            test_labels.append(sequence_labels)


    return train_sequences, train_labels, test_sequences, test_labels



In [52]:
X_train, y_train, X_test, y_test = prepare_data_lstm(df)


/tmp/ipykernel_5459/3562934620.py:251: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  'videos', 'comments', 'likes']].groupby('brand').apply(lambda x: x.iloc[:,1:].isna().sum()/len(x))
/tmp/ipykernel_5459/3562934620.py:60: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data[f'growth_rate_{column}'] = data.groupby(brand_column)[column].pct_change() * 100
/tmp/ipykernel_5459/3562934620.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment 

In [53]:
df = pd.read_csv(DATA_PATH, sep=";")
train, test = preprocess_data(df)

/tmp/ipykernel_5459/3562934620.py:139: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  'videos', 'comments', 'likes']].groupby('brand').apply(lambda x: x.iloc[:,1:].isna().sum()/len(x))
/tmp/ipykernel_5459/3562934620.py:60: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data[f'growth_rate_{column}'] = data.groupby(brand_column)[column].pct_change() * 100
/tmp/ipykernel_5459/3562934620.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment 

In [57]:
for k, v in train.isna().sum().items():
    print(k, v)


brand 0
date 0
followers 0
pictures 0
videos 0
comments 0
likes 0
domicile_country_name_Australia 0
domicile_country_name_Belgium 0
domicile_country_name_Brazil 0
domicile_country_name_Canada 0
domicile_country_name_China 0
domicile_country_name_Denmark 0
domicile_country_name_France 0
domicile_country_name_Germany 0
domicile_country_name_Italy 0
domicile_country_name_Japan 0
domicile_country_name_Mexico 0
domicile_country_name_Netherlands 0
domicile_country_name_New Zealand 0
domicile_country_name_Philippines 0
domicile_country_name_Poland 0
domicile_country_name_Singapore 0
domicile_country_name_Spain 0
domicile_country_name_Sweden 0
domicile_country_name_Switzerland 0
domicile_country_name_United Kingdom of Great Britain and Northern Ireland 0
domicile_country_name_United States of America 0
domicile_country_name_nan 0
engagement 0
engagement_rate 0
engagement_rate_per_post 0
video_picture_ratio 231935
rolling_avg_engagement 0
ewma_engagement 0
growth_rate_comments 231935
growth_rat